In [ ]:
import pandas as pd
data = pd.read_csv("train_rename.csv")
data['date'] = pd.to_datetime(data['date'])
data.head()

In [ ]:
from tqdm import trange

prefix = ["home_", "away_"]
Rolling_Average = [
"batting_batting_avg_10RA"
,"batting_onbase_perc_10RA"
,"batting_onbase_plus_slugging_10RA"
,"batting_leverage_index_avg_10RA"
,"batting_RBI_10RA"
,"pitching_earned_run_avg_10RA"
,"pitching_SO_batters_faced_10RA"
,"pitching_H_batters_faced_10RA"
,"pitching_BB_batters_faced_10RA"]

for abbr in trange(30):
    for season in range(2016, 2024):
        for pre in prefix:
                for RA in Rolling_Average:
                    RA = pre + RA
                    fliter1 = data[pre + "team_abbr"] == abbr
                    filter2 = data["season"] == season
                    temp = data[(fliter1 & filter2)][["date", RA]]
                    temp = temp.sort_values("date").reset_index()
                    
                    j = 0
                    for j in range(temp.shape[0]):
                        if not pd.isna(temp[RA][j]):
                            break
                    for i in range(j):
                        data.loc[temp["index"][i], RA] = temp[RA][j]


                    for i in range(j, temp.shape[0]):
                        if pd.isna(temp[RA][i]):
                            if i == temp.shape[0] - 1:
                                data.loc[temp["index"][i], RA] = data.loc[temp["index"][i-1], RA]
                            elif pd.isna(temp[RA][i+1]):
                                data.loc[temp["index"][i], RA] = data.loc[temp["index"][i-1], RA]
                            else:
                                data.loc[temp["index"][i], RA] = (data.loc[temp["index"][i-1], RA] + data.loc[temp["index"][i+1], RA]) / 2

In [ ]:
from tqdm import trange

prefix = ["home_", "away_"]
suffix = ["_mean", "_std", "_skew"]
Team_Seasonal = [
"team_errors"
,"team_spread"
,"team_wins"
,"batting_batting_avg"
,"batting_onbase_perc"
,"batting_onbase_plus_slugging"
,"batting_leverage_index_avg"
,"batting_wpa_bat"
,"batting_RBI"
,"pitching_earned_run_avg"
,"pitching_SO_batters_faced"
,"pitching_H_batters_faced"
,"pitching_BB_batters_faced"
,"pitching_leverage_index_avg"
,"pitching_wpa_def"]

for abbr in trange(30):
    for season in range(2016, 2024):
        for pre in prefix:
            for suf in suffix:
                for TS in Team_Seasonal:
                    TS = pre + TS + suf
                    fliter1 = data[pre + "team_abbr"] == abbr
                    filter2 = data["season"] == season
                    temp = data[(fliter1 & filter2)][["date", TS]]
                    temp = temp.sort_values("date").reset_index()
                    
                    j = 0
                    for j in range(temp.shape[0]):
                        if not pd.isna(temp[TS][j]):
                            break
                    for i in range(j):
                        data.loc[temp["index"][i], TS] = temp[TS][j]


                    for i in range(j, temp.shape[0]):
                        if pd.isna(temp[TS][i]):
                            if i == temp.shape[0] - 1:
                                data.loc[temp["index"][i], TS] = data.loc[temp["index"][i-1], TS]
                            elif pd.isna(temp[TS][i+1]):
                                data.loc[temp["index"][i], TS] = data.loc[temp["index"][i-1], TS]
                            else:
                                data.loc[temp["index"][i], TS] = (i * data.loc[temp["index"][i-1], TS] + (i+2) * data.loc[temp["index"][i+1], TS]) / (2*(i+1))

In [ ]:
data.to_csv("output.csv", index = False)